# Запустить модель можно в вашем notebook, нажав соответсвующие кнопки для запуска

# КАЧАЕМ ДАТАСЕТ

In [202]:
!wget -O pred_data_crop_minCSV.zip https://www.dropbox.com/s/5lkgz3zw9t78i8c/pred_data_crop_minCSV.zip?dl=0

In [203]:
 !unzip pred_data_crop_minCSV.zip

# ПОДКЛЮЧЕНИЕ БИБИЛИОТЕК

In [204]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Normalization
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomCrop, CenterCrop, RandomZoom, RandomContrast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
from tensorflow.keras.applications.vgg16 import preprocess_input as VGG16_ResNet50_preprocess
import matplotlib.pyplot as plt

%matplotlib inline 
from PIL import Image

# ОБРАБОТКА ДАННЫХ

In [205]:
w = 256
h = 256

In [206]:
def pred(img):
    i = Image.open(img)
    i = i.resize((256, 256), Image.ANTIALIAS)
    return np.array(i)

In [207]:
train = pd.read_csv('train.csv')
train

In [208]:
x_train = train["Названия"].apply(pred)
x_train = np.array([np.array(val,dtype=np.float32) for val in x_train])

In [209]:
x_train=VGG16_ResNet50_preprocess(x_train)

In [210]:
y_train = train.iloc[:,1:]
y_train

# РАСШИРЕНИЕ ДАННЫХ

In [211]:
data_augmentation = Sequential()
data_augmentation.add(RandomFlip("vertical", input_shape=(256,256,3)))
data_augmentation.add(RandomRotation(0.2))
data_augmentation.add(RandomZoom(0.3))
data_augmentation.add(RandomCrop(128, 128))

# Предобученная сеть

In [212]:
count_klass = y_train.shape[1]

In [213]:
ResNet50_net = ResNet50(weights='imagenet', 
                  include_top=False, 
                  input_shape=(128, 128, 3))

In [214]:
ResNet50_net.trainable = False

# АРХИТЕКТУРА

In [215]:
model = Sequential()

model.add(data_augmentation)

model.add(ResNet50_net)

model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(count_klass, activation='sigmoid'))

In [216]:
model.summary()

# ОБУЧЕНИЕ И ДОБАВЛЕНИЕ МЕТРИКИ F1

In [217]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + 
    K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m])

# CALLBACK

In [218]:
ch_p=ModelCheckpoint('best_model.h5', 
                    monitor='val_f1_m',
                    mode='max',
                    verbose=1, 
                    save_best_only=True,
                    )

In [220]:
model.fit(x_train, y_train,
          validation_split=0.2, 
          epochs=200, 
          batch_size=10,
          callbacks=[ch_p],
          verbose=1)

# Оцениваем качество обучения

In [221]:
test = pd.read_csv('test.csv')
x_test = test["Названия"].apply(pred)
x_test = np.array([np.array(val,dtype=np.float32) for val in x_test])
x_test=VGG16_ResNet50_preprocess(x_test)
y_test = test.iloc[:,1:]
y_test

In [222]:
 model.load_weights("best_model.h5")

In [ ]:
predictions = model.predict(x_test)

In [225]:
model.evaluate(x_test, y_test)

# Тонкая настройка

In [227]:
ResNet50_net.trainable = True
trainable = False
for layer in ResNet50_net.layers:
    layer.trainable = False
layer.trainable = trainable

In [228]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-5), 
              metrics=[f1_m])

In [229]:
history = model.fit(x_train, y_train,
                    validation_split=0.2,
                    batch_size=10,
                    callbacks=[ch_p],
                    epochs=25)

In [230]:
model.load_weights("best_model.h5")

# Оцениваем качество обучения после тонкой настройки

In [231]:
predictions = model.predict(x_test)

In [232]:
model.evaluate(x_test,y_test)

In [233]:
n = 127
plt.imshow(x_test[n], cmap=plt.cm.binary)
plt.show()
# print(predictions[n].round())
# print(np.round(predictions[n], 3))
arr =[]
for i in predictions[n]:
    if i >= 0.5:
      arr.append(1)
    else:
      arr.append(0)
print(arr)
print(y_test.iloc[n])